In [1]:
!pip install scikit-learn --user --upgrade

In [3]:
import os
import pickle
import cv2
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 함수들
def calculate_angles(vectors):
    angles = []
    for v in vectors:
        v_norm = np.linalg.norm(v)
        if v_norm == 0:
            angles.append([0, 0, 0])
            continue
        
        v = v / v_norm
        angle_x = np.arccos(np.clip(v[0], -1.0, 1.0))
        angle_y = np.arccos(np.clip(v[1], -1.0, 1.0))
        angle_z = np.arccos(np.clip(v[2], -1.0, 1.0)) 
        angles.append([angle_x, angle_y, angle_z])
    return np.array(angles)

def min_max_scaling(v):
    """min_max 스케일링 함수"""
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(v)
    transformed_v = scaler.transform(v)
    return transformed_v

def normalize_vector(v):
    """벡터를 정규화하는 함수."""
    # v_squared = v**2
    # v = v_squared * np.sign(v)
    
    v_norm = np.linalg.norm(v, axis=1)[:, np.newaxis]
    normalized_v = v / v_norm
    return normalized_v

# 손가락 끝 조인트에 가중치를 적용하는 함수
def apply_joint_weights(data, weights):
    if len(data) != 60:
        return data  # 데이터 길이가 60이 아닌 경우 가중치 적용하지 않음
    weighted_data = data.copy()
    for idx, weight in weights.items():
        if idx * 3 + 2 < len(weighted_data):  # 인덱스 범위 체크
            weighted_data[idx * 3] *= weight
            weighted_data[idx * 3 + 1] *= weight
            weighted_data[idx * 3 + 2] *= weight
    return weighted_data

"""
---------------------------------------------------------------------------------------------------
"""

print("start...")

mp_hands = mp.solutions.hands

# Hands 모델 초기화
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

# 데이터 디렉토리 경로
DATA_DIR = './data'

# 데이터와 라벨을 저장할 리스트 초기화
data = []
labels = []

# # 손가락 끝 조인트와 가중치 설정
finger_tip_joints = [9]
weights = {joint: 2.0 for joint in finger_tip_joints}

print("loading...")
# 데이터 디렉토리 내의 각 디렉토리에 대해 처리
for dir_ in os.listdir(DATA_DIR):
    print("start dataset making " + dir_)
    # 디렉토리만 처리
    if os.path.isdir(os.path.join(DATA_DIR, dir_)):
        # 디렉토리 내의 각 이미지 파일에 대해 처리
        for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
            # 이미지 파일인지 확인
            if img_path.endswith(('.jpg', '.jpeg', '.png')):
                # 손 위치 데이터를 저장할 리스트 초기화
                data_aux = []

                # 이미지 읽기
                img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # 손 위치 검출
                results = hands.process(img_rgb)
        
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        joint = np.zeros((21,3))
                        for i, hl in enumerate(hand_landmarks.landmark):
                            joint[i] = [hl.x, hl.y, hl.z]
                        vec1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19,6,10],:]   #관절 추가
                        vec2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,10,14],:]

                        # 벡터로 변경
                        v = vec2-vec1

                        # 정규화 혹은 각도 전처리 (하나 사용)
                        # v = normalize_vector(v)  # 정규화
                        # v = min_max_scaling(v)   # min-max 스케일링
                        v = calculate_angles(v)    # 각도

                        for i in range(len(v)):
                            data_aux.append(v[i][0])
                            data_aux.append(v[i][1])
                            data_aux.append(v[i][2])


                    # 데이터 길이 확인 및 패딩 (데이터 길이가 밖이면 수정 [현재 66개])
                    if len(data_aux) < 66:
                        data_aux.extend([0] * (66 - len(data_aux)))
                        print("lower")
                    elif len(data_aux) > 66:
                        data_aux = data_aux[:66]
                        print("over")
                        print(img_path)

                    # 데이터와 라벨 리스트에 추가
                    data.append(data_aux)
                    labels.append(int(dir_))

# 데이터를 pickle 파일로 저장
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)

print("finish")
print(data_aux)
"""
                        실험
                        
                        v2v = v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]
                        v1v = v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:]
                        for i in range(15):
                            v1=np.array(v1v[i])
                            v2=np.array(v2v[i])
                            v1_norm = np.linalg.norm(v1)
                            v2_norm = np.linalg.norm(v2)

                            if v1_norm==0 or v2_norm==0:
                                angles.append(0)
                                continue

                            v1 = v1 / v1_norm
                            v2 = v2 / v2_norm

                            cos_theta = np.dot(v1, v2)
                            cos_theta = np.clip(cos_theta, -1.0, 1.0)

                            angle_rad = np.arccos(cos_theta) 

                            # 외적을 이용하여 방향성 고려 (2D 또는 3D 벡터에 따라 다름)
                            cross_product = np.cross(v1, v2)
                            if len(cross_product) == 3 and cross_product[2] < 0:
                                angle_rad = -angle_rad
    
                            # 각도 계산 (도 단위)
                            angle_deg = np.degrees(angle_rad)
    
                            data_aux.append(angle_deg)
                        """

start...
loading...
start dataset making .gitignore
start dataset making .ipynb_checkpoints
start dataset making 0
start dataset making 1
start dataset making 10
start dataset making 11
start dataset making 12
start dataset making 13
start dataset making 14
start dataset making 15
start dataset making 16
start dataset making 17
start dataset making 18
start dataset making 19
start dataset making 2
start dataset making 20
start dataset making 21
start dataset making 22
start dataset making 23
start dataset making 24
start dataset making 25
start dataset making 26
over
1 (438).jpg
start dataset making 27
start dataset making 28
start dataset making 29
start dataset making 3
start dataset making 30
start dataset making 4
start dataset making 5
start dataset making 6
start dataset making 7
start dataset making 8
start dataset making 9
start dataset making 라벨링.txt
finish
[0.9278526394869556, 0.23560562324581125, 0.9880883487095334, 0.7814805608738924, 0.4239029311154528, 0.8308466332606698,

'\n                        실험\n                        \n                        v2v = v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]\n                        v1v = v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:]\n                        for i in range(15):\n                            v1=np.array(v1v[i])\n                            v2=np.array(v2v[i])\n                            v1_norm = np.linalg.norm(v1)\n                            v2_norm = np.linalg.norm(v2)\n\n                            if v1_norm==0 or v2_norm==0:\n                                angles.append(0)\n                                continue\n\n                            v1 = v1 / v1_norm\n                            v2 = v2 / v2_norm\n\n                            cos_theta = np.dot(v1, v2)\n                            cos_theta = np.clip(cos_theta, -1.0, 1.0)\n\n                            angle_rad = np.arccos(cos_theta) \n\n                            # 외적을 이용하여 방향성 고려 (2D 또는 3D 벡터에 따라 다름)\n                     